# MNIST: image classification by CNN
## Introduction
This notebook describes the structure of the different convolutional neural networks (CNNs) that were trained on the famous MNIST handwritten digit data.

## The problem
From the training data set of 42000 handwritten digits from 0 to 9, I want to build and train a CNN that can predict the value of a given set of digits to a high degree of accuracy. This will require designing multiple CNNs and comparing the results.

## Libraries

In [1]:
import sys, os

import numpy as np #Linear algebra etc
import pandas as pd #Data handling

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf

DATA_DIR = os.path.join(os.getcwd(), os.pardir, "data")
SRC_DIR = os.path.join(os.getcwd(), os.pardir, "src")
sys.path.append(SRC_DIR)

# Additional modules from project src directory
from data.read_data import read_data_reshaped

## Reading the data
The `read_data_reshaped()` function is in the project `src/data` directory.

In [2]:
X, y = read_data_reshaped(os.path.join(DATA_DIR, "raw", "train.csv"))

Loaded 42000 images.


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.1, random_state=42)

In [9]:
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255

## 3-layer CNN
This is intended to provide a basic 3-layer CNN, primarily for the purposes of ensuring everything is set up properly. However, this particular network was found to perform well with audio classification problems (Huzaifah 2017) so might actually be quite accurate here, too.

The basic structure is:
1. 2D convolution layer, 180 3x3 filters and ReLU activation
2. Max pooling, 4x4
3. Dropout layer, 50%
4. Fully-connected layer, 800 units and ReLU activation
5. Dropout layer, 50%
6. Output logits layer, 10 units

In [10]:
def cnn_3_layer(features, labels, mode):
    # Input layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # Convultional layer
    conv_1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=180,
        kernel_size=[3, 3],
        activation=tf.nn.relu)
    
    # Pooling layer
    pooling_1 = tf.layers.max_pooling2d(
        inputs=conv_1,
        pool_size=[4, 4],
        strides=4)
    
    # Dropout layer 1
    dropout_1 = tf.layers.dropout(
        inputs=pooling_1,
        rate=0.5)
    
#     dropout_1_flat = tf.reshape(dropout_1, [-1, 7 * 7 * 64])
    images_flat = tf.contrib.layers.flatten(dropout_1)
    # Dense layer
    dense = tf.layers.dense(
        inputs=images_flat,
        units=800,
        activation=tf.nn.relu)
    
    # Dropout layer 2
    dropout_2 = tf.layers.dropout(
        inputs=dense,
        rate=0.5)
    
    # Logits layer
    logits = tf.layers.dense(
        inputs=dropout_2,
        units=10)
    
    # Predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [15]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_3_layer, model_dir=os.path.join(os.getcwd(), os.pardir, "models"))

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=5000,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/david/Documents/data/learn/mnist/notebooks/../models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f15adfa5d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/david/Documents/data/learn/mnist/not

INFO:tensorflow:loss = 2.2922466, step = 1
INFO:tensorflow:probabilities = [[0.00408221 0.00036623 0.00590567 0.00563521 0.00461785 0.00551034
  0.0000688  0.00012921 0.96285206 0.01083246]
 [0.00001706 0.00189262 0.00001712 0.00003262 0.95453876 0.00025911
  0.00020865 0.00519297 0.00065894 0.03718206]
 [0.00000163 0.00000104 0.00000244 0.0000816  0.00003862 0.00000093
  0.00000011 0.98398054 0.00000981 0.01588328]
 [0.00016337 0.00000012 0.00000063 0.00002968 0.00359843 0.00004979
  0.00001876 0.95444036 0.00013812 0.0415607 ]
 [0.9393596  0.00016006 0.04229154 0.01101729 0.00036693 0.00066766
  0.00040034 0.00067002 0.00218775 0.00287881]
 [0.9800827  0.00005394 0.00078026 0.00011893 0.001287   0.00054457
  0.01584673 0.00003227 0.00112453 0.00012894]
 [0.99852943 0.00000002 0.00003529 0.00004189 0.0000015  0.00029812
  0.0000394  0.00000419 0.00092655 0.00012363]
 [0.00011478 0.00001478 0.00056492 0.00006623 0.02131307 0.00024345
  0.9775733  0.00000926 0.00005407 0.00004606]
 [0.0

INFO:tensorflow:global_step/sec: 8.39515
INFO:tensorflow:probabilities = [[0.00087475 0.0000003  0.00009985 0.00000101 0.00001209 0.00000222
  0.9987733  0.00000015 0.0002357  0.00000066]
 [0.00084141 0.00000001 0.0000557  0.00021137 0.00052273 0.00028735
  0.0000197  0.0074234  0.00136412 0.9892742 ]
 [0.00097649 0.00833925 0.9771673  0.00020941 0.00126224 0.00001208
  0.0035799  0.00001487 0.00831168 0.0001268 ]
 [0.00047516 0.5314991  0.00927829 0.00564023 0.37939635 0.00130864
  0.0012451  0.00075529 0.0449767  0.02542517]
 [0.00003144 0.00000104 0.00005887 0.00014899 0.00194349 0.00033749
  0.00000736 0.00327584 0.00160898 0.99258643]
 [0.9386855  0.00002987 0.01257006 0.00021303 0.001211   0.00002224
  0.00193158 0.03513991 0.00434586 0.00585087]
 [0.00083448 0.00095237 0.00657936 0.00018653 0.00033293 0.00018171
  0.9881563  0.00000133 0.00277044 0.00000464]
 [0.00017966 0.00113301 0.02152778 0.7143887  0.00003229 0.1940872
  0.00095629 0.00002372 0.06764777 0.0000235 ]
 [0.0000

INFO:tensorflow:loss = 0.14965883, step = 101 (11.913 sec)
INFO:tensorflow:probabilities = [[0.99990845 0.         0.0000209  0.0000007  0.00000001 0.00001018
  0.00001349 0.00000001 0.00004624 0.00000004]
 [0.00002083 0.00000036 0.00002732 0.9430661  0.00000036 0.05475827
  0.00000064 0.00000057 0.00211143 0.0000141 ]
 [0.00001085 0.00000001 0.00005768 0.9977302  0.00000006 0.00209119
  0.00000004 0.00000292 0.00007613 0.00003085]
 [0.00397258 0.00064323 0.00011422 0.00390913 0.05831734 0.0097954
  0.00053205 0.24177215 0.00411565 0.67682827]
 [0.00021806 0.00000004 0.00004868 0.0006821  0.00206683 0.00132796
  0.00000269 0.010186   0.0004163  0.9850514 ]
 [0.0000012  0.00000006 0.00050531 0.9967391  0.00000256 0.00202774
  0.00000029 0.00000047 0.00071108 0.00001217]
 [0.9997389  0.         0.00000983 0.00000104 0.00000002 0.00006396
  0.00012265 0.00000005 0.00006311 0.00000038]
 [0.00022112 0.9982436  0.00023778 0.00002461 0.0002131  0.00001845
  0.00031081 0.00007034 0.00065259 0.

INFO:tensorflow:global_step/sec: 8.43203
INFO:tensorflow:probabilities = [[0.00000126 0.         0.0000004  0.00168329 0.00000005 0.9980204
  0.00000003 0.00000114 0.0000122  0.00028122]
 [0.00000135 0.00001739 0.99911505 0.00055754 0.00000268 0.00023691
  0.00000157 0.00000001 0.00006491 0.00000262]
 [0.00000534 0.00000003 0.00032574 0.98421854 0.0000001  0.01408436
  0.00000003 0.00000061 0.00092236 0.00044286]
 [0.00006833 0.998021   0.00015351 0.00004292 0.00058754 0.00003979
  0.00048539 0.00002737 0.00056377 0.00001043]
 [0.00001115 0.00000533 0.00012123 0.00002634 0.00052363 0.00094731
  0.9982338  0.00000033 0.00012128 0.00000952]
 [0.00181814 0.00000119 0.00256277 0.00013945 0.01093927 0.06801489
  0.00632075 0.00391637 0.00772002 0.8985671 ]
 [0.00249395 0.0345469  0.15678231 0.0005498  0.43199298 0.00366424
  0.30359992 0.00026284 0.06535704 0.0007501 ]
 [0.99945956 0.         0.00007118 0.0000006  0.         0.00015915
  0.00003289 0.00000001 0.00027108 0.0000056 ]
 [0.0000

...

INFO:tensorflow:global_step/sec: 8.26053
INFO:tensorflow:probabilities = [[0.         0.99999976 0.         0.         0.         0.00000001
  0.         0.00000029 0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.00000001 1.         0.         0.         0.         0.
  0.         0.00000003 0.         0.        ]
 [0.         0.         0.         0.99999976 0.         0.00000022
  0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.00000003]
 [0.         0.         1.         0.00000003 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.         0.000001   0.000001   0.00002049 0.         0.
  0.         0.9999776  0.00000001 0.        ]
 [0.         0.00000026 0.00001122 0.         0.       

INFO:tensorflow:loss = 0.0008852099, step = 4801 (12.108 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.0006316  0.         0.00000273 0.00000026 0.00000006 0.
  0.         0.00000221 0.         0.9993631 ]
 [0.9999999  0.         0.00000007 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.00000005 0.00061492 0.00000002 0.00000671 0.00002487
  0.         0.00005309 0.00336118 0.99593914]
 [0.00000002 0.         0.00000001 0.         0.99783486 0.
  0.         0.00216455 0.0000006  0.00000005]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.00000001]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.00000

INFO:tensorflow:global_step/sec: 8.15812
INFO:tensorflow:probabilities = [[0.         0.         0.9999895  0.00001019 0.         0.
  0.         0.         0.00000022 0.        ]
 [0.         0.         0.         0.         0.99576724 0.
  0.         0.00423277 0.00000001 0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.00000003 0.        ]
 [0.         0.         0.         0.00000002 0.         0.
  0.         0.         1.         0.00000001]
 [0.         0.00000007 0.9999999  0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.00000001 0.
  0.         0.         0.00000002 1.        ]
 [0.         0.         0.         0.         0.9997547  0.
  0.         0.00001062 0.00000443 0.00023021]
 [0.         0.00000001 0.00000001 0.00000004 0.00000004 0.
  0.       

INFO:tensorflow:loss = 0.00028933107, step = 4901 (12.255 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         0.         0.         0.0000004
  0.99999964 0.         0.00000002 0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.00000001]
 [0.00000183 0.9999852  0.0000002  0.00000005 0.00000034 0.
  0.         0.00000673 0.0000056  0.00000001]
 [0.99998844 0.         0.00000718 0.         0.         0.
  0.         0.         0.         0.00000446]
 [0.9999994  0.         0.00000049 0.         0.         0.
  0.         0.         0.0000001  0.00000006]
 [0.00000005 0.9999865  0.00000308 0.00000001 0.00000986 0.
  0.         0.0000002  0.00000024 0.        ]
 [0.         0.         0.         0.00000275 0.         0.99999714
  0.         0.         0.00000001 0.0000001 ]
 [1.         0.         0.        

INFO:tensorflow:Saving checkpoints for 5000 into /home/david/Documents/data/learn/mnist/notebooks/../models/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0002812131.


Changes steps to 3000.

In [12]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_3_layer, model_dir=os.path.join(os.getcwd(), os.pardir, "models"))

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=3000,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/david/Documents/data/learn/mnist/notebooks/../models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f15a7693320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/david/Documents/data/learn/mnist/not

INFO:tensorflow:loss = 2.290106, step = 1
INFO:tensorflow:probabilities = [[0.00049055 0.0002379  0.99510807 0.00293198 0.0000001  0.00001047
  0.00000418 0.00000092 0.00121569 0.00000006]
 [0.00004281 0.9962212  0.00018893 0.0002687  0.0000716  0.00019072
  0.0005117  0.00000649 0.00248114 0.0000167 ]
 [0.00005443 0.00001305 0.00006128 0.00053907 0.01494657 0.00028859
  0.00000653 0.03062838 0.00072737 0.9527348 ]
 [0.00001561 0.9961461  0.00031789 0.00043493 0.00009959 0.00010793
  0.00009282 0.00006551 0.00260823 0.00011135]
 [0.00157082 0.12940913 0.02107123 0.00298931 0.3876058  0.03772159
  0.00093164 0.04047927 0.34617868 0.03204252]
 [0.00012888 0.00000996 0.00108144 0.00002827 0.01568939 0.00052467
  0.98165524 0.00000087 0.00080242 0.00007893]
 [0.0000018  0.00000003 0.00001245 0.00001242 0.0000001  0.00000018
  0.00000001 0.9999125  0.00000623 0.00005433]
 [0.00449204 0.00000096 0.00022923 0.00000729 0.9882051  0.00037538
  0.00171564 0.0021573  0.00118261 0.00163454]
 [0.00

INFO:tensorflow:global_step/sec: 8.06698
INFO:tensorflow:probabilities = [[0.00000133 0.         0.9998683  0.00000451 0.         0.00000211
  0.00000005 0.00000225 0.00011272 0.0000088 ]
 [0.00003946 0.         0.00000779 0.00007568 0.00101419 0.00154119
  0.00000457 0.00283226 0.00390157 0.9905833 ]
 [0.0000673  0.00001203 0.00073074 0.0001357  0.00004927 0.00576505
  0.00005829 0.00001566 0.9926703  0.0004957 ]
 [0.0000323  0.00000002 0.00003924 0.00215628 0.0005198  0.99411964
  0.00009237 0.0001612  0.00034862 0.00253064]
 [0.00002817 0.00000205 0.69466895 0.00490848 0.00012208 0.00031603
  0.00002908 0.28678825 0.0004776  0.0126594 ]
 [0.00002249 0.00004129 0.00043874 0.8787948  0.00015564 0.05110325
  0.00003189 0.00018399 0.05078232 0.01844557]
 [0.9988739  0.         0.00000935 0.00004226 0.00000304 0.00006886
  0.00016105 0.00001526 0.00071874 0.00010751]
 [0.00022776 0.0008557  0.00008859 0.00698796 0.0179009  0.00960552
  0.00001869 0.08724368 0.0128212  0.86424994]
 [0.000

INFO:tensorflow:loss = 0.103517085, step = 101 (12.399 sec)
INFO:tensorflow:probabilities = [[0.00000016 0.00000066 0.9999795  0.00000324 0.00000024 0.00000002
  0.0000001  0.00000006 0.0000159  0.00000015]
 [0.00045401 0.00107508 0.00004743 0.01000982 0.00083048 0.9518346
  0.00018783 0.00228853 0.02152193 0.01175011]
 [0.00025506 0.9917685  0.00040857 0.00033738 0.00548179 0.00000118
  0.00016639 0.00026652 0.00125477 0.00005982]
 [0.0000075  0.00000609 0.00000123 0.00000281 0.98278975 0.00000622
  0.00000334 0.00579075 0.00025227 0.01114007]
 [0.00047153 0.00000037 0.00005448 0.6002241  0.00000141 0.39374056
  0.00008395 0.0000025  0.00433199 0.00108914]
 [0.00001133 0.00000002 0.00000198 0.00000011 0.99842525 0.00000004
  0.00013665 0.00003904 0.00004002 0.00134565]
 [0.00001191 0.00007775 0.00089084 0.00116749 0.00070347 0.00008115
  0.00000066 0.95449924 0.00051329 0.04205436]
 [0.00371141 0.00000327 0.00407435 0.58571315 0.00019919 0.36963952
  0.00545407 0.00007755 0.02799032 0

INFO:tensorflow:global_step/sec: 8.02751
INFO:tensorflow:probabilities = [[0.00007288 0.00000011 0.00000517 0.00001341 0.00000122 0.9982147
  0.00000427 0.00029905 0.00019645 0.0011927 ]
 [0.00001004 0.00000002 0.00021956 0.97054416 0.         0.02883323
  0.00000112 0.00000007 0.00036798 0.00002381]
 [0.00000082 0.00000205 0.00016942 0.00026956 0.00001962 0.0000014
  0.00000032 0.9893351  0.00036248 0.00983912]
 [0.0000008  0.00000265 0.9997347  0.00000937 0.00015329 0.00000013
  0.00001115 0.00000008 0.000086   0.00000166]
 [0.00000005 0.00048706 0.9994049  0.00009592 0.         0.00000001
  0.00000003 0.00000306 0.00000883 0.00000017]
 [0.00000115 0.00000001 0.9997602  0.0000457  0.00000105 0.00000007
  0.00000056 0.00011447 0.00007038 0.00000641]
 [0.94882655 0.         0.00032445 0.00003026 0.00000001 0.02377947
  0.00111889 0.00002597 0.01155037 0.01434404]
 [0.00000419 0.00000025 0.00004898 0.9908997  0.00000001 0.00886704
  0.000001   0.00000117 0.00008696 0.00009075]
 [0.00049

...

INFO:tensorflow:global_step/sec: 8.50086
INFO:tensorflow:probabilities = [[0.00000038 0.         0.00000002 0.00000057 0.0000001  0.
  0.         0.00000613 0.         0.99999285]
 [0.         0.00000002 0.00000032 0.00000005 0.0006867  0.
  0.         0.00004838 0.00000323 0.9992613 ]
 [0.999995   0.         0.00000164 0.         0.         0.
  0.00000001 0.00000055 0.         0.00000271]
 [0.         0.         0.         0.         0.0000001  0.
  0.9999999  0.         0.         0.        ]
 [0.         0.00000414 0.00000008 0.00000085 0.00000254 0.00000001
  0.00000042 0.         0.9999845  0.00000756]
 [0.         0.00011296 0.00000011 0.00000006 0.00000018 0.00000117
  0.00000674 0.00000002 0.9998758  0.00000297]
 [0.         0.         0.         0.         0.0000009  0.
  0.         0.00000002 0.         0.99999905]
 [0.00004436 0.         0.00000009 0.         0.00020764 0.
  0.9997478  0.         0.00000001 0.00000006]
 [0.         0.00000022 0.         0.         0.0000067

INFO:tensorflow:loss = 0.0005430533, step = 2801 (11.764 sec)
INFO:tensorflow:probabilities = [[0.         0.00000047 0.00000001 0.00917439 0.00000139 0.9783393
  0.00000327 0.         0.01248031 0.00000084]
 [0.99996674 0.         0.00003267 0.         0.         0.
  0.00000004 0.00000003 0.00000055 0.00000004]
 [0.         0.00000017 0.00000241 0.00000061 0.         0.
  0.         0.99999666 0.00000005 0.00000014]
 [0.00000006 0.99999964 0.00000003 0.         0.         0.
  0.         0.00000027 0.         0.        ]
 [0.00021005 0.00000318 0.         0.00000003 0.00002136 0.
  0.         0.9997532  0.00000001 0.00001216]
 [0.00000001 0.0000001  0.         0.00000003 0.00000008 0.
  0.         0.99999154 0.00000009 0.00000811]
 [0.00000001 0.9999962  0.00000002 0.         0.         0.
  0.         0.00000382 0.         0.        ]
 [0.00018379 0.0000205  0.00000186 0.         0.00000047 0.00000091
  0.9997924  0.         0.00000002 0.00000002]
 [0.         0.00000006 0.00000299 

INFO:tensorflow:global_step/sec: 8.50234
INFO:tensorflow:probabilities = [[0.         0.         0.         0.         0.         0.
  1.         0.         0.         0.        ]
 [0.9999945  0.         0.00000512 0.00000001 0.         0.
  0.         0.00000002 0.00000006 0.00000027]
 [0.         0.         0.         0.         0.00000018 0.
  0.99999976 0.         0.         0.        ]
 [0.         0.00000002 0.         0.         0.9999981  0.
  0.         0.0000016  0.00000019 0.        ]
 [0.         0.         0.         0.00000057 0.00000059 0.
  0.         0.00000244 0.00000005 0.9999964 ]
 [0.         0.         0.9999907  0.00000851 0.00000072 0.
  0.         0.         0.00000002 0.00000006]
 [0.         0.         0.         0.         0.         0.00000155
  0.00000003 0.         0.99999845 0.        ]
 [0.         0.00000002 0.         0.00001116 0.         0.
  0.         0.99998796 0.         0.00000088]
 [0.         0.00000029 0.         0.00000019 0.9808594  0.0000

INFO:tensorflow:loss = 0.0065693506, step = 2901 (11.762 sec)
INFO:tensorflow:probabilities = [[0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.9999914  0.         0.00000003 0.         0.         0.
  0.00000688 0.         0.         0.00000171]
 [0.         0.         0.         0.         0.9999999  0.
  0.         0.         0.00000008 0.00000002]
 [0.         0.         0.         0.00004739 0.00000004 0.99994504
  0.00000004 0.         0.00000492 0.00000253]
 [0.         0.         0.0000009  0.00000001 0.         0.
  0.         0.99999905 0.         0.00000003]
 [0.         0.         0.         0.00000001 0.00001933 0.
  0.         0.00000002 0.00000366 0.999977  ]
 [0.00000002 0.         0.         0.         0.         0.
  0.9999999  0.         0.0000001  0.        ]
 [0.         0.99999917 0.00000004 0.         0.00000043 0.
  0.         0.0000004  0.00000003 0.        ]
 [0.         0.         0.         0.00000

INFO:tensorflow:Saving checkpoints for 3000 into /home/david/Documents/data/learn/mnist/notebooks/../models/model.ckpt.
INFO:tensorflow:Loss for final step: 0.036018033.


In [16]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-18-16:38:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/david/Documents/data/learn/mnist/notebooks/../models/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-18-16:38:14
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9892857, global_step = 5000, loss = 0.04468709
{'accuracy': 0.9892857, 'loss': 0.04468709, 'global_step': 5000}
